In [57]:
import json
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import locale

locale.setlocale(locale.LC_TIME, 'fr_FR')

'fr_FR'

In [58]:
#ouverture de la connexion avec postgres

db_params = {
    "database": "job_market",
    "user": "admin",
    "password": "root",
    "host": "localhost",
    "port": "5432"
}

conn = psycopg2.connect(**db_params)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

In [59]:
# création du dataframe complet
job_offer_adzuna = pd.read_csv('../output/job_offers_adzuna.csv')
job_offer_wttj = pd.read_csv('../output/job_offers_wttj.csv')
df = pd.concat([job_offer_adzuna, job_offer_wttj], ignore_index=True)

# nettoyage rapide
df = df[~df.company.isna()].reset_index(drop=True)

#print(f"type of df.starting_date {type(df.starting_date)} et df.starting_date {df.starting_date} \n")
df = df.assign(
	#starting_date=pd.to_datetime(df.starting_date,dayfirst=True)
    starting_date=pd.to_datetime(df['starting_date'], format='%d %B %Y')
)
df['starting_date'] = df['starting_date'].dt.date

df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce', format='%Y-%m-%d')

df = df.where(pd.notnull(df), None)

In [78]:
# for the Skills table
# attention au R pour qu'il ne soit pas confondu avec une majuscule

df_skills = pd.read_csv('./skills.csv', header=None)
skills_list = df_skills.iloc[:, 0].tolist()
#print(list(skills_list))
#print(len(skills_list))
#print(type(skills_list))

for skill in skills_list:
    cur.execute("SELECT skillName FROM Skills WHERE skillName = %s;", (skill,))
    result = cur.fetchone()
    if result is None:
        cur.execute("INSERT INTO Skills (skillName) VALUES (%s);", [skill])

In [79]:
"""
for the JobOffer_Skills table
2 possiblités : 
    soit une ligne par skillId (et donc plusieurs lignes pour le même jobId) 
    soit une ligne par jobId  ligne (et donc plusieurs pour le même skillId)

pour rappel
CREATE TABLE JobOffer_Skills (
        jobOfferId INTEGER REFERENCES JobOffers,
        skillId INTEGER REFERENCES Skills(skillId),
        PRIMARY KEY (jobOfferId, skillId)
    );

Direction des flèches: il y a deux relations impliquées ici
JobOffer vers JobOffer_Skills
Skill vers JobOffer_Skills
Signification : Chaque flèche pointe vers JobOffer_Skills, indiquant qu’elle sert de table de jonction. 
Il n’y a pas de flèche directe entre JobOffer et Skills parce que leur relation est médiée à travers JobOffer_Skills. 
Un offre d’emploi peut être associée à plusieurs compétences et vice versa.

"""
def link_job_skill(cur, row, jobOfferId):
    description = row['description']
    if description:
        for skill in skills_list:
            if skill in description:
                cur.execute("SELECT skillId FROM Skills WHERE skillName = %s;", [skill])
                skill_id  = cur.fetchone()
                if skill_id:
                    cur.execute("INSERT INTO JobOffer_Skills (jobofferid,skillid) VALUES (%s, %s);", (jobOfferId,skill_id,))

In [67]:
def get_or_create_company(cur, company_data):
    company = eval(company_data)
    cur.execute("SELECT companyId FROM Companies WHERE companyName = %s;", (company['name'],))
    result = cur.fetchone()
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO Companies (companyName, location, sector, information) VALUES (%s, %s, %s, %s) RETURNING companyId;",
                    (company['name'], company.get('location'), company.get('sector'), ''))
        return cur.fetchone()[0]

In [68]:
def get_or_create_source(cur, source_name):
    cur.execute("SELECT sourceId FROM Sources WHERE sourceName = %s;", (source_name,))
    result = cur.fetchone()
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO Sources (sourceName) VALUES (%s) RETURNING sourceId;", (source_name,))
        return cur.fetchone()[0]

In [80]:
# itération sur chaque ligne du dataframe pour intégrer chaque annonce à JobOffers

for _, row in df.iterrows():
    company_id = get_or_create_company(cur, row['company'])
    source_id = get_or_create_source(cur, row['source'])

    job_offer_data = (
        row['title'],
        company_id,
        row['salary'],
        row['remote_type'],
        row['contract_type'],
        row['starting_date'] if not pd.isnull(row['starting_date']) else None,
        row['location'],
        row['require_experience'],
        row['education'],
        row['description'],
        row['profil_experience'],
        row['publication_date'].date() if not pd.isnull(row['publication_date']) else None,
        row['url_direct_offer'],
        source_id,
    )
    cur.execute("INSERT INTO JobOffers (title, companyId, salary, remoteType, contractType, startingDate, location, requiredExp, education, descriptions, profilExp, publicationDate, jobLink, sourceId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING jobOfferId;", job_offer_data)
    jobOfferId = cur.fetchone()
    link_job_skill(cur,row, jobOfferId)

Description du poste Apprenez un métier d’avenir en alternance avec OpenClassrooms. Nous recherchons un Data Engineer en Alternance (H/F) pour notre partenaire du secteur de l'événementiel. Cette offre ne s’adresse qu’aux candidats à l’alternance qui effectuent leur formation avec OpenClassrooms ou souhaitent s’inscrire chez OpenClassrooms pour leur alternance. Seules les candidatures répondant à ces critères seront étudiées. Avec OpenClassrooms, vous apprendrez un métier avec une pédagogie mêl… 

Nous recherchons un( e ) DATA ENGINEER en CDI pour rejoindre notre équipe à ROUEN Rendez-vous au salon Les Emplois en Seine les 14 et 15 mars 2024 au parc expo de Rouen pour rencontrer les recruteurs. Consultez de nombreuses autres offres sur normandie-emploi.fr 

Qui sommes-nous ? NEXTON c'est avant tout une entreprise qui accompagne ses clients dans leur transformation digitale. Tous les jours, nous travaillons avec des grands comptes et des pures players (SNCF, Orange, BNP PARIBAS…). Nous 

In [39]:
#clôre la connexion avec postgres

conn.commit()
cur.close()
conn.close()